In [1]:
# Read all QALD-8 Queries
# For each question
    # Extract all URIs
    # Find sameAs links to DE, FR, ES dbpedia
    # If links exist in same language for all URIs then;
        # generate sparql for all languages
        # write the sparql like: "query_<language-abbr>": { "sparql": <sparql> }
# Store the new QALD File

In [14]:
# Load QALD8 questions

# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://raw.githubusercontent.com/ag-sc/QALD/master/7/data/qald-7-train-multilingual.json"
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())
  
# print the json response
print(data_json['dataset']['id'])

qald-7-train-multilingual


In [12]:
import time
from SPARQLWrapper import SPARQLWrapper, JSON
from string import Template

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql_template = 'SELECT ?l WHERE { <$uri> owl:sameAs ?l . FILTER(regex(str(?l), "http://$lang.dbpedia.org/" )) }'
def fetch_links(uri, lang):
    links_set = set()
    # form the sparql
    query = Template(sparql_template).substitute(uri=uri, lang=lang)
    # query
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    #print('Result size:',len(results["results"]["bindings"]))

    for result in results["results"]["bindings"]:
        links_set.add(result['l']['value'])
    
    time.sleep(0.1)
    return links_set

# function to check and return links for URIs in a given language
def check_links(uri_list, lang):
    missing_links = False
    # For each uri look for language specific links
    link_map = {}
    for uri in uri_list:
        links_set = fetch_links(uri, lang)
        if len(links_set) == 0:
            missing_links = True
        link_map[uri] = links_set
    return (missing_links, link_map)

In [14]:
# Test Block
# print(check_links({'http://dbpedia.org/resource/Barack_Obama','http://dbpedia.org/resource/Donald_Trump'},'de'))

In [ ]:
# Convert SPARQL
def convert_sparql(sparql, prefix_tuples, uris, lang):
    # fetch the links for tuples
    # fetch the links for uris
    # if missing links then return null
    # create unique combinations for links
    # for each combination, create sparql
    # return all the sparql queries
def get_all_sparql(sparql, prefix_tuples, uris):
    # for each langauge fetch the sparql queries
    # return the queries

In [13]:
import re
prefix_map = {}
uris = set()
for question in data_json['questions']:
    sparql_uris = set()
    sparql = question['query']['sparql']    
   # extract all prefixes to make a map
    prefix_tuples=re.findall('PREFIX\s+([a-zA-Z0-9]+):\s+<(.*?)>',sparql, re.IGNORECASE)
    for entry in prefix_tuples:
        prefix_map[entry[0]] = entry[1]
    #print(prefix_map)
    # extract all prefix mentions
    prefix_mentions = re.findall('{?[\s\t\n\r]+([\w\d]+):([^<\s]+)',sparql, re.IGNORECASE)
    #print(prefix_mentions)
    # join the local name with prefix and store it in uri set
    
    for entry in prefix_mentions:
        sparql_uris.add(prefix_map[entry[0]]+entry[1])
    # print(uris)
    # extract all direct uri mentions
    extra_uris = re.findall('<(.*?)>',sparql, re.IGNORECASE)
    sparql_uris.update(extra_uris)
    # Generate and save language specific SPARQL queries 
    uris.update(sparql_uris)
print('Total', len(uris) , 'unique URIs found.')

Total 381 unique URIs found.
